In [ ]:
## preprocessing

## libraries
import glob
import random
import pretty_midi
import IPython
import numpy as np
from tqdm import tnrange, tqdm
from tqdm import notebook as tqdm
from random import shuffle, seed
import numpy as np

import unicodedata
import re
import numpy as np
import os
import io
import time

## how to unzip a folder of midi files in the notebook before running preprocessing
#unzips the folder
import zipfile
with zipfile.ZipFile("arabic_tribal_rhythms_ZIP.zip","r") as zip_ref:
    zip_ref.extractall()#where you want the file to be unzipped, empty means I want it in the same folder?


In [85]:
#1 

#i want this to take a folder of midi files and produce a list
class preprocessing_class:
    
    
    def __init__(self, folder, batch_size, seq_len):
        self.folder = folder
        self.batch_size = batch_size
        self.seq_len = seq_len


   #2
    def generate_dict_time_notes(self, list_all_midi, start_index=0, fs=30, use_tqdm=True):
        """ Generate map (dictionary) of music ( in index ) to piano_roll (in np.array)

        Parameters
        ==========
        list_all_midi : list
            List of midi files
        batch_size : int
          A number of music in one batch
        start_index : int
          The start index to be batched in list_all_midi
        fs : int
          Sampling frequency of the columns, i.e. each column is spaced apart
            by ``1./fs`` seconds.
        use_tqdm : bool
          Whether to use tqdm or not in the function

        Returns
        =======
        dictionary of music to piano_roll (in np.array)"""
        assert len(list_all_midi) >= self.batch_size
        list_time_notes = []
        process_tqdm_midi = tqdm.tqdm(range(start_index, len(list_all_midi))) if use_tqdm else range(start_index,  min(len(list_all_midi)))
        for i in process_tqdm_midi:
            midi_file_name = list_all_midi[i]
            if use_tqdm:
                process_tqdm_midi.set_description("Processing {}".format(midi_file_name))
            try: # Handle exception on malformat MIDI files
                midi_pretty_format = pretty_midi.PrettyMIDI(midi_file_name)
                piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
                piano_roll = piano_midi.get_piano_roll(fs=fs)
                list_time_notes.append(piano_roll) 
                print(i)
            except Exception as e:
                print(e)
                print("broken file : {}".format(midi_file_name))
                pass
        return list_time_notes
    
    def preprocess_midi(self):
        list_all_midi = glob.glob(self.folder)
        seed(666)
        shuffle(list_all_midi)
        list_time_notes = self.generate_dict_time_notes(list_all_midi, start_index=0, fs=30, use_tqdm=True)
        return list_time_notes


#
p = preprocessing_class("arabic_tribal_rhythms/arabic_tribal_rhythms/*.mid", 1, 8)
q= p.preprocess_midi()